In [1]:
%install '.package(path: "$cwd/FastaiNotebook_08a_heterogeneous_dictionary")' FastaiNotebook_08a_heterogeneous_dictionary

Installing packages:
	.package(path: "/home/ubuntu/fastai_docs/dev_swift/FastaiNotebook_08a_heterogeneous_dictionary")
		FastaiNotebook_08a_heterogeneous_dictionary
With SwiftPM flags: []
Working in: /tmp/tmpwwmggsmj/swift-install
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 3.86s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebook_08a_heterogeneous_dictionary' (13 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


## Load data

In [2]:
import FastaiNotebook_08a_heterogeneous_dictionary
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [3]:
// export
import Path
import TensorFlow

Will train on imagenette when it's possible without OOM.

In [4]:
//let path = downloadImagette()

In [5]:
//let il = ItemList(fromFolder: path, extensions: ["jpeg", "jpg"])
//let sd = SplitData(il, fromFunc: {grandParentSplitter(fName: $0, valid: "val")})
//var (procItem,procLabel) = (NoopProcessor<Path>(),CategoryProcessor())
//let sld = SplitLabeledData(sd, fromFunc: parentLabeler, procItem: &procItem, procLabel: &procLabel)
//var rawData = sld.toDataBunch(itemToTensor: pathsToTensor, labelToTensor: intsToTensor)
//let data = transformData(rawData, tfmItem: { openAndResize(fname: $0, size: 128) })

In [6]:
let data = mnistDataBunch(flat: true)

In [7]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [8]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

## Stateful optimizer

In [9]:
//export
//Expandable enum to have easier names than LearningRate.self.
public struct HyperParams {
    public static let lr = LearningRate.self
}

In [10]:
let hd = HeterogeneousDictionary()

In [11]:
hd[HyperParams.lr]

0.4


In [12]:
//export
open class StatDelegate {
    open var name: String { return "" }
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    public func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) { }
}

//export
open class StepDelegate {
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) { }
}

In [13]:
//export
public class StatefulOptimizer<Model: Layer>
    where Model.AllDifferentiableVariables == Model.CotangentVector{
    public var configs: [HeterogeneousDictionary]
    public var splitFunc: (Int) -> Int
    public var states: [String: Model.AllDifferentiableVariables]
    public var statDelegates: [StatDelegate]
    public var stepDelegates: [StepDelegate]
    public init(
        for model: __shared Model,
        stepDelegates: [StepDelegate],
        statDelegates: [StatDelegate],
        configs: [HeterogeneousDictionary],
        splitFunc: @escaping (Int) -> Int
    ) {
        self.configs = Array(repeating: HeterogeneousDictionary(), count: configs.count)
        states = [:]
        for stepDelegate in stepDelegates {
            for i in self.configs.indices { self.configs[i].merge(stepDelegate.defaultConfig) { (_, new) in new } }
        }
        for statDelegate in statDelegates {
            for i in self.configs.indices { self.configs[i].merge(statDelegate.defaultConfig) { (_, new) in new } }
            states[statDelegate.name] = Model.AllDifferentiableVariables.zero
        }
        for i in 0..<configs.count {
            self.configs[i].merge(configs[i]) { (_, new) in new }
        }
        self.stepDelegates = stepDelegates
        self.statDelegates = statDelegates
        self.splitFunc = splitFunc
    }
        
    func update(
        _ model: inout Model.AllDifferentiableVariables,
        along direction: Model.CotangentVector
    ) {
        for (i,kp) in model.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self).enumerated() {
            var grad = direction[keyPath: kp]
            var state = states.mapValues(){$0[keyPath: kp]}
            var config = configs[splitFunc(i)]
            for statDelegate in statDelegates {
                statDelegate.update(
                    state: &state,
                    for: model[keyPath: kp],
                    along: grad,
                    config: &config
                )
            }
            for n in states.keys { states[n]![keyPath: kp] = state[n]! }
            for stepDelegate in stepDelegates {
                stepDelegate.update(
                    param: &model[keyPath: kp],
                    along: &grad,
                    state: state,
                    config: &config
                )
            }
            configs[splitFunc(i)] = config
        }
    }
}

Conformance to the optimizer protocol

In [14]:
extension StatefulOptimizer: Optimizer{
    public var learningRate: Float {
        get { return configs.last![HyperParams.lr] } 
        set { 
            for i in configs.indices {self.configs[i][HyperParams.lr] = newValue }
        }
    }
    public var learningRates: [Float] {
        get {
            var res: [Float] = []
            for config in configs {res.append(config[HyperParams.lr])}
            return res
        }
        set { 
            for i in configs.indices {self.configs[i][HyperParams.lr] = newValue[i] } 
        }
    }
}

Convenience init when there are no param groups

In [15]:
//export
extension StatefulOptimizer{
    public convenience init (for model: __shared Model,
                             stepDelegates: [StepDelegate],
                             statDelegates: [StatDelegate],
                             config: HeterogeneousDictionary) {
        self.init(for: model,
                  stepDelegates: stepDelegates,
                  statDelegates: statDelegates,
                  configs: [config],
                  splitFunc: { _ in return 0 })
    }
}

In [16]:
//export
public class SGDStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= direction * config[HyperParams.lr]
    }
}

Test discriminative learning rates/freezing

In [17]:
func splitFunc(_ a: Int) -> Int { return a < 2 ? 0 : 1 }

In [18]:
var configs = [HeterogeneousDictionary(HyperParams.lr, 0.0), HeterogeneousDictionary(HyperParams.lr, 0.01)]
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [SGDStep()],
        statDelegates: [],
        configs: configs,
        splitFunc: splitFunc)
}

In [19]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [20]:
let params = learner.model.allDifferentiableVariables

In [21]:
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[   0.024127113,   -0.051024936,    -0.02905424,     0.05214011,  -0.0081088785,
    -0.04773733,   -0.011530449,   -0.041617233,    0.012290093,    -0.07545146,
   -0.044788096,   0.0012412205,    -0.05976695,    0.033521265,    0.058854535,
     0.06558435,    0.011890215,    -0.06555809,     0.06486427, -0.00036192028,
    -0.07384039,     0.04204685,   -0.023879135,    -0.05933991,   -0.027547346,
   -0.005728743,   -0.010772705,    0.039684568,     0.08066905,   -0.047627512,
     0.07396252,   -0.028902106, -0.00046165715,    -0.03514182,    0.048072033,
    -0.07193122,   -0.024367213,   -0.050203785,    -0.05424294,   -0.004000294,
   -0.042725958,     0.07228505,    -0.00131118,     0.06805487,     -0.0327277,
    -0.06750118,    0.051023297,     0.06483211,    -0.07080606,    0.055634987]
0.0
[ 0.29121563, -0.22275923, -0.22873153,  0.18006407,   0.2591383,  0.25828883,   0.2882882,
 -0.10005783,   0.0800995, 0.038988043]
0.0


In [22]:
learner.fit(2)

Epoch 0: [1.2609471, 0.6292]                                                    
Epoch 1: [1.0236813, 0.7011]                                                    


In [23]:
let params = learner.model.allDifferentiableVariables
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[   0.024127113,   -0.051024936,    -0.02905424,     0.05214011,  -0.0081088785,
    -0.04773733,   -0.011530449,   -0.041617233,    0.012290093,    -0.07545146,
   -0.044788096,   0.0012412205,    -0.05976695,    0.033521265,    0.058854535,
     0.06558435,    0.011890215,    -0.06555809,     0.06486427, -0.00036192028,
    -0.07384039,     0.04204685,   -0.023879135,    -0.05933991,   -0.027547346,
   -0.005728743,   -0.010772705,    0.039684568,     0.08066905,   -0.047627512,
     0.07396252,   -0.028902106, -0.00046165715,    -0.03514182,    0.048072033,
    -0.07193122,   -0.024367213,   -0.050203785,    -0.05424294,   -0.004000294,
   -0.042725958,     0.07228505,    -0.00131118,     0.06805487,     -0.0327277,
    -0.06750118,    0.051023297,     0.06483211,    -0.07080606,    0.055634987]
0.0
[  0.19557741, -0.021629084,  -0.27866045,   0.18392797,   0.38770422, 0.0007976555,
 -0.036791235,   0.14926054,  -0.15633267,   0.42067903]
-0.105407976


In [24]:
//export
public struct WeightDecayKey: HetDictKey { public static var defaultValue: Float = 0 }

public extension HyperParams {
    static let wd = WeightDecayKey.self
}

public class WeightDecay: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param *= 1 - config[HyperParams.lr] * config[HyperParams.wd]
    }
}

In [25]:
//export
public class L2Regularization: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        direction += config[HyperParams.wd] * param
    }
}

In [26]:
//export
//Expandable enum to have tab completes/typo-proof for state variable names.
public struct StateKeys {
    public static let avgGrad = "averageGrad"
}

In [27]:
//export

public struct Momentum: HetDictKey { public static var defaultValue: Float = 0.9 }
public struct MomentumDampening: HetDictKey, Equatable { public static var defaultValue: Float = 0.1 }
public extension HyperParams {
    static let mom = Momentum.self
    static let momDamp = MomentumDampening.self
}

class AverageGrad: StatDelegate {
    let dampened: Bool
    init(dampened: Bool = false) { self.dampened = dampened }
    override var name: String { return StateKeys.avgGrad }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.avgGrad]! *= config[HyperParams.mom]
        config[HyperParams.momDamp] = 1.0 - (dampened ? config[HyperParams.mom] : 0.0)
        state[StateKeys.avgGrad]! += config[HyperParams.momDamp] * direction
    }
}

In [28]:
//export
public class MomentumStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= state[StateKeys.avgGrad]! * config[HyperParams.lr]
    }
}

In [30]:
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [MomentumStep()],
        statDelegates: [AverageGrad()],
        config: HeterogeneousDictionary(HyperParams.lr, 0.01))
}

In [31]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [32]:
learner.fit(2)

Epoch 0: [0.16618632, 0.9513]                                                   
Epoch 1: [0.12381144, 0.9613]                                                   


In [46]:
//export
public struct SquareMomentum: HetDictKey { public static var defaultValue: Float = 0.99 }
public struct SquareMomentumDampening: HetDictKey { public static var defaultValue: Float = 0.99 }

public extension HyperParams {
    static let ²mom = Momentum.self
    static let ²momDamp = MomentumDampening.self
}

public extension StateKeys {
    static let avgSqr = "averageSquaredGrad"
}

class AverageSquaredGrad: StatDelegate {
    let dampened: Bool
    init(dampened: Bool = true) { self.dampened = dampened }
    override var name: String { return StateKeys.avgSqr }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.avgSqr]! *= config[HyperParams.²mom]
        config[HyperParams.²momDamp] = 1.0 - (dampened ? config[HyperParams.²mom] : 0.0)
        state[StateKeys.avgSqr]! += config[HyperParams.²momDamp] * direction.squared()
    }
}

In [34]:
//export
public extension StateKeys {
    static let step = "stepCount"
}

class StepCount: StatDelegate {
    override var name: String { return StateKeys.step }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.step]! += 1.0
    }
}

In [35]:
//export
public struct Epsilon: HetDictKey { public static var defaultValue: Float = 1e-5 }
public extension HyperParams {
    static let eps = Epsilon.self
}

In [36]:
//export
public class AdamStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let step = state[StateKeys.step]!
        let (mom,damp) = (config[HyperParams.mom],config[HyperParams.momDamp])
        let debias1 = damp * (1 - pow(mom, step)) / (1 - mom)
        let num = debias1 * state[StateKeys.avgGrad]!
        
        let (²mom,²damp) = (config[HyperParams.²mom],config[HyperParams.²momDamp])
        let debias2 = ²damp * (1 - pow(²mom, step)) / (1 - ²mom)
        let denom = sqrt(state[StateKeys.avgSqr]!/debias2) + config[HyperParams.eps]
        
        param -= config[HyperParams.lr] * num / denom
    }
}

In [37]:
func opt(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [AdamStep()], 
        statDelegates: [AverageGrad(dampened: true), AverageSquaredGrad(), StepCount()], 
        config: HeterogeneousDictionary(HyperParams.lr, 0.01))
}

In [38]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [39]:
learner.fit(2)

Epoch 0: [0.15804851, 0.9539]                                                   
Epoch 1: [0.12964107, 0.9604]                                                   


In [40]:
class LambStep: StepDelegate {
    override var defaultConfig: HeterogeneousDictionary {
        return HeterogeneousDictionary(HyperParams.eps, 1e-6, HyperParams.wd, 0.0)
    }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let stepCount = state[StateKeys.step]!
        let (mom,damp) = (config[HyperParams.mom],config[HyperParams.momDamp])
        let debias1 = damp * (1 - pow(mom, stepCount)) / (1 - mom)
        let num = debias1 * state[StateKeys.avgGrad]!
        
        let (²mom,²damp) = (config[HyperParams.²mom],config[HyperParams.²momDamp])
        let debias2 = ²damp * (1 - pow(²mom, stepCount)) / (1 - ²mom)
        let denom = sqrt(state[StateKeys.avgSqr]!/debias2) + config[HyperParams.eps]
        
        let step = num / denom + config[HyperParams.wd] * param
        let r1 = sqrt((param * param).mean())
        let r2 = sqrt((step * step).mean())
        let factor = min(r1 / r2, Float(10.0))
        param -= config[HyperParams.lr] * factor * step
    }
}

### Making convenienc functions

In [45]:
// export
public func SGDOpt<Model>(lr: Float, mom: Float = 0.9, wd: Float = 0.0, dampening: Bool = false
                         ) -> ((Model) -> StatefulOptimizer<Model>) {
    var steppers = (mom != 0) ? [MomentumStep()] : [SGDStep()]
    if wd != 0 { steppers.append(WeightDecay()) }
    let stats = (mom != 0) ? [AverageGrad(dampened: dampening)] : []
    var config = HeterogeneousDictionary(HyperParams.lr, lr)
    if mom != 0 { config[HyperParams.mom] = mom }
    if wd != 0  { config[HyperParams.wd ] = wd  }
    return { model -> StatefulOptimizer<Model> in
        return StatefulOptimizer(for: model, stepDelegates: steppers, statDelegates: stats, config: config)
    }
}

In [48]:
// export
public func AdamOpt<Model>(lr: Float, mom: Float = 0.9, beta: Float=0.99, wd: Float = 0.0, eps: Float = 1e-5
                         ) -> ((Model) -> StatefulOptimizer<Model>) {
    var steppers: [StepDelegate] = [AdamStep()]
    if wd != 0 { steppers.append(WeightDecay()) }
    let stats = [AverageGrad(dampened: true), AverageSquaredGrad(), StepCount()]
    var config = HeterogeneousDictionary(HyperParams.lr, lr)
    config[HyperParams.mom] = mom
    config[HyperParams.²mom] = beta
    config[HyperParams.eps] = eps
    if wd != 0  { config[HyperParams.wd ] = wd  }
    return { model -> StatefulOptimizer<Model> in
        return StatefulOptimizer(for: model, stepDelegates: steppers, statDelegates: stats, config: config)
    }
}

## Export

In [50]:
notebookToScript(fname: (Path.cwd / "09_optimizer.ipynb").string)